### Chatbot with Conversational History

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model='Gemma2-9b-It', groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000220904DEBA0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000220904DF770>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('user', 'Hi my name is Yuvraj and Im a GenAI engineer'),
    ("ai", "Hi Yuvraj, its nice to meet you. Thats fascinating, being a GenAI engineer is cutting edge and important work. What kind of projects are you working on? Im always eager to learn more about the advancements in the field.  Perhaps you could tell me about some of the challenges and rewards you encounter in your role?"),
    ('user', "Whats my name and what do i do?")
])

chain=prompt|model
chain.invoke({})

AIMessage(content="Hi Yuvraj, it's nice to meet you!\n\nThat's fascinating, being a GenAI engineer is cutting-edge and important work. What kind of projects are you working on?\n\nI'm always eager to learn more about the advancements in the field.  Perhaps you could tell me about some of the challenges and rewards you encounter in your role?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 22, 'total_tokens': 101, 'completion_time': 0.143636364, 'prompt_time': 0.001326271, 'queue_time': 0.256360298, 'total_time': 0.144962635}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--01566863-66ad-4910-950d-8f6659514586-0', usage_metadata={'input_tokens': 22, 'output_tokens': 79, 'total_tokens': 101})